In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import joblib
import shap
from datetime import datetime

import xgboost as xgb
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, 
    roc_curve, precision_recall_curve, average_precision_score,
    accuracy_score, precision_score, recall_score, f1_score
)
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
import time

# Ignore warnings
warnings.filterwarnings('ignore')

c:\Users\ryanf\Desktop\ML Sports Prediction\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Visualization settings\

plt.style.use('default')
sns.set_palette("husl")

In [4]:
# Configuration and random seeds

RANDOM_STATE = 42
TARGET_COLUMN = 'injury_next_14_days'
MODEL_NAME = 'nba_injury_predictor_v1'

# Random seeds for reproducibility
np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)

print(f"Configuration and etc:")
print(f"- Target: {TARGET_COLUMN}")
print(f"- Random State: {RANDOM_STATE}")
print(f"- Model Name: {MODEL_NAME}")
print(f"- TensorFlow version: {tf.__version__}")
print(f"- GPU available: {len(tf.config.list_physical_devices('GPU')) > 0}")

Configuration and etc:
- Target: injury_next_14_days
- Random State: 42
- Model Name: nba_injury_predictor_v1
- TensorFlow version: 2.19.0
- GPU available: False


In [5]:
# Loads all processed data
# Training data (SMOTE balanced + feature selected)
X_train = pd.read_csv('../data/processed/X_train_final.csv')
y_train = pd.read_csv('../data/processed/y_train_final.csv').squeeze()

# Validation data (feature selected)
X_val = pd.read_csv('../data/processed/X_validation_final.csv')
y_val = pd.read_csv('../data/processed/y_validation_final.csv').squeeze()

# Test data (feature selected)
X_test = pd.read_csv('../data/processed/X_test_final.csv')
y_test = pd.read_csv('../data/processed/y_test_final.csv').squeeze()

print(f"Data loaded successfully:")
print(f"- Training: {X_train.shape} features, {len(y_train)} samples")
print(f"- Validation: {X_val.shape} features, {len(y_val)} samples") 
print(f"- Test: {X_test.shape} features, {len(y_test)} samples")

# Loads metadata and configuration
# Selected features list
selected_features = joblib.load('../data/processed/selected_features.pkl')
print(f"- Selected features: {len(selected_features)}")

# Class weights for handling imbalance
class_weights = joblib.load('../data/processed/class_weights.pkl')
print(f"- Class weights: {class_weights}")

# Preprocessing configuration
preprocessing_config = joblib.load('../data/processed/preprocessing_config.pkl')
print(f"- Preprocessing config loaded")

# Feature selection results
feature_selection_results = joblib.load('../data/processed/feature_selection_results.pkl')
print(f"- Feature selection metadata loaded")

# Split information for validation
split_info = joblib.load('../data/processed/split_info.pkl')
print(f"- Data split validation loaded")

# Data validation and consistency checks
# Check feature consistency
assert list(X_train.columns) == selected_features, "Training features don't match selected features"
assert list(X_val.columns) == selected_features, "Validation features don't match selected features"  
assert list(X_test.columns) == selected_features, "Test features don't match selected features"
print("- Feature consistency across all splits")

# Checks target distributions
train_positive_rate = y_train.mean()
val_positive_rate = y_val.mean()
test_positive_rate = y_test.mean()

print(f"\nTarget distribution validation:")
print(f"- Training positive rate: {train_positive_rate:.1%} (after SMOTE)")
print(f"- Validation positive rate: {val_positive_rate:.1%}")
print(f"- Test positive rate: {test_positive_rate:.1%}")

# Checks for missing values
train_missing = X_train.isnull().sum().sum()
val_missing = X_val.isnull().sum().sum()
test_missing = X_test.isnull().sum().sum()

assert train_missing == 0, f"Training data has {train_missing} missing values"
assert val_missing == 0, f"Validation data has {val_missing} missing values"
assert test_missing == 0, f"Test data has {test_missing} missing values"
print("- No missing values in any split")

# Verifies data types
assert X_train.dtypes.apply(lambda x: x.kind in 'biufc').all(), "Non-numeric features in training"
assert X_val.dtypes.apply(lambda x: x.kind in 'biufc').all(), "Non-numeric features in validation"
assert X_test.dtypes.apply(lambda x: x.kind in 'biufc').all(), "Non-numeric features in test"
print("- All features are numeric")

print("\nAll data validation checks passed!")

# Feature statistics
print(f"\nFeature Statistics (Training Data):")
print(f"- Mean range: {X_train.mean().min():.3f} to {X_train.mean().max():.3f}")
print(f"- Std range: {X_train.std().min():.3f} to {X_train.std().max():.3f}")
print(f"- Min values: {X_train.min().min():.3f} to {X_train.min().max():.3f}")
print(f"- Max values: {X_train.max().min():.3f} to {X_train.max().max():.3f}")

# Checks for potential scaling issues
features_need_scaling = (X_train.std() > 10).sum()
print(f"- Features with std > 10: {features_need_scaling} (may need scaling)")

# Targets class balance verification
print(f"\nClass Balance Check:")
print(f"- Training: {y_train.value_counts().to_dict()}")
print(f"- Validation: {y_val.value_counts().to_dict()}")
print(f"- Test: {y_test.value_counts().to_dict()}")

# Sample feature names by category
print(f"\nSample Features by Type:")
workload_features = [f for f in selected_features if any(x in f for x in ['_7d', '_30d', 'load'])]
fatigue_features = [f for f in selected_features if any(x in f for x in ['fatigue', 'rest', 'back_to_back'])]
context_features = [f for f in selected_features if any(x in f for x in ['age', 'bmi', 'position'])]

print(f"- Workload features ({len(workload_features)}): {workload_features[:3]}...")
print(f"- Fatigue features ({len(fatigue_features)}): {fatigue_features[:3]}...")
print(f"- Context features ({len(context_features)}): {context_features[:3]}...")


# Data preprocessing for modeling
# Feature scaling
# RobustScaler to handle outliers better than StandardScaler
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrames for easier handling
X_train_scaled = pd.DataFrame(X_train_scaled, columns=selected_features, index=X_train.index)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=selected_features, index=X_val.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=selected_features, index=X_test.index)

print(f"- Features scaled using RobustScaler")
print(f"  - Training scaled shape: {X_train_scaled.shape}")
print(f"  - Scaled feature stats: mean≈{X_train_scaled.mean().mean():.3f}, std≈{X_train_scaled.std().mean():.3f}")

# Converts to numpy arrays for TensorFlow
X_train_tf = X_train_scaled.values.astype(np.float32)
X_val_tf = X_val_scaled.values.astype(np.float32)
X_test_tf = X_test_scaled.values.astype(np.float32)
y_train_tf = y_train.values.astype(np.float32)
y_val_tf = y_val.values.astype(np.float32)
y_test_tf = y_test.values.astype(np.float32)

print(f"- Data converted to TensorFlow format")
print(f"  - Input shape: {X_train_tf.shape}")
print(f"  - Target shape: {y_train_tf.shape}")
print(f"  - Data types: {X_train_tf.dtype}, {y_train_tf.dtype}")

# Stores scaler for later use
joblib.dump(scaler, f'../data/processed/{MODEL_NAME}_scaler.pkl')
print(f"- Scaler saved for deployment")

print("Data loaded and prepared for modeling stage")
print(f"Prepared to build TensorFlow model with {X_train_tf.shape[1]} features")

Data loaded successfully:
- Training: (6975, 34) features, 6975 samples
- Validation: (2567, 34) features, 2567 samples
- Test: (589, 34) features, 589 samples
- Selected features: 34
- Class weights: {0: 0.512264982373678, 1: 20.88323353293413}
- Preprocessing config loaded
- Feature selection metadata loaded
- Data split validation loaded
- Feature consistency across all splits

Target distribution validation:
- Training positive rate: 2.4% (after SMOTE)
- Validation positive rate: 3.0%
- Test positive rate: 1.0%
- No missing values in any split
- All features are numeric

All data validation checks passed!

Feature Statistics (Training Data):
- Mean range: -0.000 to 1117.278
- Std range: 0.029 to 280.307
- Min values: -11.500 to 23.750
- Max values: 0.170 to 1757.000
- Features with std > 10: 5 (may need scaling)

Class Balance Check:
- Training: {0: 6808, 1: 167}
- Validation: {0: 2490, 1: 77}
- Test: {0: 583, 1: 6}

Sample Features by Type:
- Workload features (7): ['total_actions

In [6]:
# Validation

print("Data shapes after all preprocessing:")
print(f"  - X_train: {X_train_tf.shape}")
print(f"  - y_train: {y_train_tf.shape}")
print(f"  - X_val: {X_val_tf.shape}")
print(f"  - y_val: {y_val_tf.shape}")
print(f"  - X_test: {X_test_tf.shape}")
print(f"  - y_test: {y_test_tf.shape}")

print(f"\nClass distribution summary:")
print(f"  - Training: {np.bincount(y_train_tf.astype(int))} (ratio: {(y_train_tf == 0).sum()/(y_train_tf == 1).sum():.1f}:1)")
print(f"  - Validation: {np.bincount(y_val_tf.astype(int))} (ratio: {(y_val_tf == 0).sum()/(y_val_tf == 1).sum():.1f}:1)")
print(f"  - Test: {np.bincount(y_test_tf.astype(int))} (ratio: {(y_test_tf == 0).sum()/(y_test_tf == 1).sum():.1f}:1)")

print(f"\nClass weights for model: {class_weights}")

print(f"\nReady")
print(f"   - Features: {X_train_tf.shape[1]}")
print(f"   - Training samples: {X_train_tf.shape[0]:,}")
print(f"   - Target: {TARGET_COLUMN}")
print(f"   - Model: {MODEL_NAME}")

Data shapes after all preprocessing:
  - X_train: (6975, 34)
  - y_train: (6975,)
  - X_val: (2567, 34)
  - y_val: (2567,)
  - X_test: (589, 34)
  - y_test: (589,)

Class distribution summary:
  - Training: [6808  167] (ratio: 40.8:1)
  - Validation: [2490   77] (ratio: 32.3:1)
  - Test: [583   6] (ratio: 97.2:1)

Class weights for model: {0: 0.512264982373678, 1: 20.88323353293413}

Ready
   - Features: 34
   - Training samples: 6,975
   - Target: injury_next_14_days
   - Model: nba_injury_predictor_v1


In [7]:
# Feature scaling

scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print(f"- Training scaled shape: {X_train_scaled.shape}")
print(f"- Feature stats after scaling: mean={X_train_scaled.mean():.3f}, std={X_train_scaled.std():.3f}")


- Training scaled shape: (6975, 34)
- Feature stats after scaling: mean=0.147, std=4.644


# Logistic Regression Model

### Overview

The model shows strong performance on the training set but experiences degradation on validation and test sets, suggesting potential overfitting. The low precision scores across all sets indicate challenges with the severe class imbalance in injury prediction. The feature importance reveals interesting patterns where some high activity metrics actually appear protective, possibly due to conditioning effects or survivor bias in the data.

### Configuration

- Random state for reproducibility
- Class weights to handle class imbalance
- L2 regularization with penalty strength of 1.0
- Maximum 1000 iterations to ensure convergence
- LBFGS solver optimized for small datasets

### Performance

**Training Set Performance**
- Accuracy: 74.7%
- Precision: 6.7%
- Recall: 74.3%
- F1-Score: 12.3%
- ROC AUC: 83.0%
- PR AUC: 18.5%

**Confusion Matrix:**
- True Negatives: 5,088
- False Positives: 1,720
- False Negatives: 43
- True Positives: 124

**Validation Set Performance**
- Accuracy: 75.7%
- Precision: 5.7%
- Recall: 45.5%
- F1-Score: 10.1%
- ROC AUC: 64.5%
- PR AUC: 9.8%

**Confusion Matrix:**
- True Negatives: 1,907
- False Positives: 583
- False Negatives: 42
- True Positives: 35

**Test Set Performance**
- Accuracy: 47.7%
- Precision: 0.7%
- Recall: 33.3%
- F1-Score: 1.3%
- ROC AUC: 37.2%
- PR AUC: 0.9%

**Confusion Matrix:**
- True Negatives: 279
- False Positives: 304
- False Negatives: 4
- True Positives: 2

### Top K Risk Prediction Analysis

- **Top 5%**: Captures 14.3% of all injuries with 8.6% precision
- **Top 10%**: Captures 27.3% of all injuries with 8.2% precision
- **Top 15%**: Captures 32.5% of all injuries with 6.5% precision
- **Top 20%**: Captures 37.7% of all injuries with 5.7% precision

### Feature Importance

**Features that Increase Injury Risk:**
1. **Fatigue Score** (coefficient: 1.818) - Strong positive predictor
2. **Total Actions** (coefficient: 1.417) - Higher activity increases risk
3. **Shooting Load 30d** (coefficient: 1.175) - Cumulative shooting strain
4. **Is Low Performance** (coefficient: 1.132) - Poor performance indicator
5. **Defensive Load 30d** (coefficient: 0.905) - Defensive workload impact
6. **Total Actions 30d** (coefficient: 0.437) - Monthly activity accumulation

**Features that Decrease Injury Risk:**
1. **Is Back to Back** (coefficient: -1.844) - Counterintuitive protective factor
2. **Cumulative Actions 30d** (coefficient: -1.467) - Higher cumulative load protective
3. **Rebounds** (coefficient: -0.978) - Rebounding activity protective
4. **Missed Shots** (coefficient: -0.951) - More missed shots lower risk
5. **Total Shot Attempts** (coefficient: -0.608) - Higher shooting volume protective
6. **Game Day of Week** (coefficient: -0.502) - Certain days lower risk
7. **Shooting Efficiency** (coefficient: -0.403) - Better efficiency protective
8. **Contact Usage Rate** (coefficient: -0.376) - Higher contact usage protective
9. **Is Weekend Game** (coefficient: -0.369) - Weekend games lower risk


In [8]:
# Logisitc Regression
logistic_model = LogisticRegression(
    random_state=RANDOM_STATE,
    class_weight=class_weights,  # Handle class imbalance
    penalty='l2',                # L2 regularization
    C=1.0,                      # Regularization strength (will tune later)
    max_iter=1000,              # Ensure convergence
    solver='lbfgs'              # Good for small datasets
)

logistic_model.fit(X_train_scaled, y_train)

# Predictions

# Predictions on all sets
y_train_pred = logistic_model.predict(X_train_scaled)
y_val_pred = logistic_model.predict(X_val_scaled)
y_test_pred = logistic_model.predict(X_test_scaled)

# Prediction probabilities
y_train_prob = logistic_model.predict_proba(X_train_scaled)[:, 1]
y_val_prob = logistic_model.predict_proba(X_val_scaled)[:, 1]
y_test_prob = logistic_model.predict_proba(X_test_scaled)[:, 1]

def evaluate_model_performance(y_true, y_pred, y_prob, dataset_name):
    """
    Model evaluation
    """
    print(f"\n{dataset_name.upper()} Performance:")
    
    # Basic metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    
    # AUC metrics
    roc_auc = roc_auc_score(y_true, y_prob)
    pr_auc = average_precision_score(y_true, y_prob)
    
    print(f"- Accuracy: {accuracy:.3f}")
    print(f"- Precision: {precision:.3f}")
    print(f"- Recall: {recall:.3f}")
    print(f"- F1-Score: {f1:.3f}")
    print(f"- ROC AUC: {roc_auc:.3f}")
    print(f"- PR AUC: {pr_auc:.3f}")
    
    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    print(f"Confusion Matrix:")
    print(f"    TN: {cm[0,0]:4d} | FP: {cm[0,1]:4d}")
    print(f"    FN: {cm[1,0]:4d} | TP: {cm[1,1]:4d}")
    
    return {
        'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1,
        'roc_auc': roc_auc, 'pr_auc': pr_auc, 'confusion_matrix': cm
    }

# Evaluates on all datasets
train_metrics = evaluate_model_performance(y_train, y_train_pred, y_train_prob, "Training")
val_metrics = evaluate_model_performance(y_val, y_val_pred, y_val_prob, "Validation")
test_metrics = evaluate_model_performance(y_test, y_test_pred, y_test_prob, "Test")


TRAINING Performance:
- Accuracy: 0.747
- Precision: 0.067
- Recall: 0.743
- F1-Score: 0.123
- ROC AUC: 0.830
- PR AUC: 0.185
Confusion Matrix:
    TN: 5088 | FP: 1720
    FN:   43 | TP:  124

VALIDATION Performance:
- Accuracy: 0.757
- Precision: 0.057
- Recall: 0.455
- F1-Score: 0.101
- ROC AUC: 0.645
- PR AUC: 0.098
Confusion Matrix:
    TN: 1907 | FP:  583
    FN:   42 | TP:   35

TEST Performance:
- Accuracy: 0.477
- Precision: 0.007
- Recall: 0.333
- F1-Score: 0.013
- ROC AUC: 0.372
- PR AUC: 0.009
Confusion Matrix:
    TN:  279 | FP:  304
    FN:    4 | TP:    2


In [9]:
# Top K Risk Prediciton Analysis

def analyze_top_k_predictions(y_true, y_prob, k_values=[5, 10, 15, 20]):
    """
    Analyzes what percentage of actual injuries are captured in top K% predictions
    """
    
    print(f"Top K Risk Prediction Performance:")
    
    # Sorts by probability (highest risk first)
    sorted_indices = np.argsort(y_prob)[::-1]
    sorted_true = y_true[sorted_indices]
    
    total_positives = y_true.sum()
    n_samples = len(y_true)
    
    for k in k_values:
        # Top k% of predictions
        top_k_size = int(n_samples * k / 100)
        top_k_true = sorted_true[:top_k_size]
        
        # Calculates capture rate
        captured_positives = top_k_true.sum()
        capture_rate = captured_positives / total_positives if total_positives > 0 else 0
        precision_at_k = captured_positives / top_k_size if top_k_size > 0 else 0
        
        print(f"    Top {k:2d}%: {capture_rate*100:5.1f}% of injuries captured, "
              f"precision = {precision_at_k*100:5.1f}%")

# Analyzes on validation set
analyze_top_k_predictions(y_val.values, y_val_prob)

Top K Risk Prediction Performance:
    Top  5%:  14.3% of injuries captured, precision =   8.6%
    Top 10%:  27.3% of injuries captured, precision =   8.2%
    Top 15%:  32.5% of injuries captured, precision =   6.5%
    Top 20%:  37.7% of injuries captured, precision =   5.7%


In [10]:
# Feature importance Analysis

# Gets feature coefficients (weights)
feature_coefficients = logistic_model.coef_[0]
feature_importance = pd.DataFrame({
    'feature': selected_features,
    'coefficient': feature_coefficients,
    'abs_coefficient': np.abs(feature_coefficients)
}).sort_values('abs_coefficient', ascending=False)

print(f"Top 15 Most Important Features:")
print(f"{'Feature':<25} {'Coefficient':<12} {'Impact':<15}")

for idx, row in feature_importance.head(15).iterrows():
    impact = "Increase Injury Risk" if row['coefficient'] > 0 else "Decrease Injury Risk"
    print(f"  {row['feature']:<25} {row['coefficient']:>10.3f}   {impact}")

Top 15 Most Important Features:
Feature                   Coefficient  Impact         
  is_back_to_back               -1.844   Decrease Injury Risk
  fatigue_score                  1.818   Increase Injury Risk
  cumulative_actions_30d        -1.467   Decrease Injury Risk
  total_actions                  1.417   Increase Injury Risk
  shooting_load_30d              1.175   Increase Injury Risk
  is_low_performance             1.132   Increase Injury Risk
  rebounds                      -0.978   Decrease Injury Risk
  missed_shots                  -0.951   Decrease Injury Risk
  defensive_load_30d             0.905   Increase Injury Risk
  total_shot_attempts           -0.608   Decrease Injury Risk
  game_day_of_week              -0.502   Decrease Injury Risk
  total_actions_30d              0.437   Increase Injury Risk
  shooting_efficiency           -0.403   Decrease Injury Risk
  contact_usage_rate            -0.376   Decrease Injury Risk
  is_weekend_game               -0.369   Decr

# Random Forest Model

### Overview

The Random Forest model demonstrates classic signs of overfitting with near perfect training performance but degradation on validation and test sets. Despite hyperparameter tuning with 162 parameter combinations, the model struggles w/ generalization. The feature importance rankings reveal fatigue score as the dominant predictor, with game timing and performance trend features playing secondary roles.

### Configuration

**Hyperparameter Grid Search:**
- 162 parameter combinations tested across 3 fold cross validation (486 total fits)
- Training time: 76.0 seconds
- Scoring metric: Average Precision (PR-AUC) for imbalanced data
- Stratified K-Fold cross-validation with 3 splits

**Best Parameters:**
- Number of estimators: 150
- Maximum depth: 15
- Minimum samples split: 50
- Minimum samples leaf: 20
- Maximum features: sqrt
- Class weight: balanced_subsample
- Best cross-validation PR-AUC: 0.1729

### Performance

**Training Set Performance**
- Accuracy: 98.5%
- Precision: 62.4%
- Recall: 96.4%
- F1-Score: 75.8%
- ROC AUC: 99.6%
- PR AUC: 81.2%

**Confusion Matrix:**
- True Negatives: 6,711
- False Positives: 97
- False Negatives: 6
- True Positives: 161

**Validation Set Performance**
- Accuracy: 96.1%
- Precision: 23.9%
- Recall: 14.3%
- F1-Score: 17.9%
- ROC AUC: 64.7%
- PR AUC: 12.1%

**Confusion Matrix:**
- True Negatives: 2,455
- False Positives: 35
- False Negatives: 66
- True Positives: 11

**Test Set Performance**
- Accuracy: 94.7%
- Precision: 3.7%
- Recall: 16.7%
- F1-Score: 6.1%
- ROC AUC: 35.6%
- PR AUC: 1.5%

**Confusion Matrix:**
- True Negatives: 557
- False Positives: 26
- False Negatives: 5
- True Positives: 1

### Top K Risk Prediction Analysis

- **Top 5%**: Captures 20.8% of all injuries with 12.5% precision
- **Top 10%**: Captures 26.0% of all injuries with 7.8% precision
- **Top 15%**: Captures 33.8% of all injuries with 6.8% precision
- **Top 20%**: Captures 40.3% of all injuries with 6.0% precision

### Feature Importance

**Top 15 Most Important Features:**

1. **Fatigue Score** (0.1538) - Dominant predictor representing player exhaustion
2. **Game Day of Week** (0.0658) - Scheduling patterns impact injury risk
3. **Current vs 14-Day Average** (0.0487) - Recent performance relative to baseline
4. **Performance Drop 7 vs 30** (0.0482) - Short-term performance decline indicator
5. **Total Actions** (0.0457) - Overall game activity level
6. **Shots vs Season Average** (0.0387) - Shooting volume relative to season norm
7. **Efficiency Trend 7d** (0.0358) - Recent efficiency trajectory
8. **Contact Usage Rate** (0.0349) - Physical contact involvement
9. **Actions Trend 7d** (0.0331) - Recent activity pattern changes
10. **Rebounds vs Season Average** (0.0325) - Rebounding relative to baseline
11. **Shooting Efficiency Decline** (0.0304) - Deterioration in shooting performance
12. **Substitution Frequency** (0.0300) - In-game replacement patterns
13. **Shooting Load 30d** (0.0282) - Cumulative shooting workload
14. **Defensive Load 30d** (0.0277) - Defensive activity accumulation
15. **Cumulative Actions 30d** (0.0277) - Monthly activity total

In [11]:
# Random Forest Model

# Hyperparameter grid
rf_param_grid = {
    'n_estimators': [50, 100, 150], 
    'max_depth': [5, 10, 15],      
    'min_samples_split': [20, 50, 100], 
    'min_samples_leaf': [5, 10, 20],    
    'max_features': ['sqrt', 0.5],      
    'class_weight': ['balanced_subsample'],
    'random_state': [RANDOM_STATE]
}

rf_base = RandomForestClassifier(
    random_state=RANDOM_STATE,
    n_jobs=-1  # Use all available cores
)

# Grid Search with Cross Validation
cv_folds = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)

rf_grid_search = GridSearchCV(
    estimator=rf_base,
    param_grid=rf_param_grid,
    cv=cv_folds,
    scoring='average_precision',  # PR-AUC for imbalanced data
    n_jobs=-1,
    verbose=1
)

start_time = time.time()
rf_grid_search.fit(X_train_scaled, y_train)
training_time = time.time() - start_time

print(f"Training completed in {training_time:.1f} seconds")
print(f"Best parameters: {rf_grid_search.best_params_}")
print(f"Best cross-validation PR-AUC: {rf_grid_search.best_score_:.4f}")

# Get the best model
rf_model = rf_grid_search.best_estimator_

# Make predictions on all sets
y_train_pred_rf = rf_model.predict(X_train_scaled)
y_val_pred_rf = rf_model.predict(X_val_scaled)
y_test_pred_rf = rf_model.predict(X_test_scaled)

# Prediction probabilities
y_train_prob_rf = rf_model.predict_proba(X_train_scaled)[:, 1]
y_val_prob_rf = rf_model.predict_proba(X_val_scaled)[:, 1]
y_test_prob_rf = rf_model.predict_proba(X_test_scaled)[:, 1]

# Evaluations
train_metrics_rf = evaluate_model_performance(y_train, y_train_pred_rf, y_train_prob_rf, "Training")
val_metrics_rf = evaluate_model_performance(y_val, y_val_pred_rf, y_val_prob_rf, "Validation")
test_metrics_rf = evaluate_model_performance(y_test, y_test_pred_rf, y_test_prob_rf, "Test")

# Top K Risk Analysis
analyze_top_k_predictions(y_val.values, y_val_prob_rf)

# Feature Importance Analysis

# Gets feature importances
feature_importance_rf = pd.DataFrame({
    'feature': selected_features,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("Top 15 Most Important Features (Random Forest):")
print(f"{'Feature':<25} {'Importance':<12}")

for idx, row in feature_importance_rf.head(15).iterrows():
    print(f"{row['feature']:<25} {row['importance']:>10.4f}")


Fitting 3 folds for each of 162 candidates, totalling 486 fits
Training completed in 117.7 seconds
Best parameters: {'class_weight': 'balanced_subsample', 'max_depth': 15, 'max_features': 'sqrt', 'min_samples_leaf': 20, 'min_samples_split': 50, 'n_estimators': 150, 'random_state': 42}
Best cross-validation PR-AUC: 0.1729

TRAINING Performance:
- Accuracy: 0.985
- Precision: 0.624
- Recall: 0.964
- F1-Score: 0.758
- ROC AUC: 0.996
- PR AUC: 0.812
Confusion Matrix:
    TN: 6711 | FP:   97
    FN:    6 | TP:  161

VALIDATION Performance:
- Accuracy: 0.961
- Precision: 0.239
- Recall: 0.143
- F1-Score: 0.179
- ROC AUC: 0.647
- PR AUC: 0.121
Confusion Matrix:
    TN: 2455 | FP:   35
    FN:   66 | TP:   11

TEST Performance:
- Accuracy: 0.947
- Precision: 0.037
- Recall: 0.167
- F1-Score: 0.061
- ROC AUC: 0.356
- PR AUC: 0.015
Confusion Matrix:
    TN:  557 | FP:   26
    FN:    5 | TP:    1
Top K Risk Prediction Performance:
    Top  5%:  20.8% of injuries captured, precision =  12.5%
    

# XGBoost Model

### Overview

The XGBoost model demonstrates more balanced performance compared to the Random Forest, avoiding extreme overfitting while maintaining reasonable generalization. The hyperparameter search across 2,187 parameter combinations resulted in a model that shows moderate performance degradation from training to test sets. The feature importance distribution is more balanced than Random Forest, w/ fatigue score remaining the top predictor but at a lower dominance level.

### Configuration

**Class Distribution Handling:**
- Negative class samples: 6,808
- Positive class samples: 167
- Scale positive weight: 40.77 (automatic class imbalance adjustment)

**Hyperparameter Grid Search:**
- 2,187 parameter combinations tested across 3-fold cross-validation (6,561 total fits)
- Scoring metric: Average Precision (PR-AUC) for imbalanced data
- Stratified K-Fold cross-validation with 3 splits
- Early stopping with 10 rounds patience

**Best Parameters:**
- Maximum depth: 3
- Learning rate: 0.1
- Number of estimators: 100
- Subsample ratio: 0.7
- Column subsample ratio: 0.7
- L1 regularization (reg_alpha): 0.5
- L2 regularization (reg_lambda): 2
- Best cross-validation PR-AUC: 0.143

### Performance

**Training Set Performance**
- Accuracy: 82.2%
- Precision: 9.7%
- Recall: 77.2%
- F1-Score: 17.2%
- ROC AUC: 88.4%
- PR AUC: 26.1%

**Confusion Matrix:**
- True Negatives: 5,607
- False Positives: 1,201
- False Negatives: 38
- True Positives: 129

**Validation Set Performance**
- Accuracy: 85.1%
- Precision: 7.5%
- Recall: 35.1%
- F1-Score: 12.4%
- ROC AUC: 69.0%
- PR AUC: 8.3%

**Confusion Matrix:**
- True Negatives: 2,158
- False Positives: 332
- False Negatives: 50
- True Positives: 27

**Test Set Performance**
- Accuracy: 72.7%
- Precision: 0.6%
- Recall: 16.7%
- F1-Score: 1.2%
- ROC AUC: 39.6%
- PR AUC: 1.1%

**Confusion Matrix:**
- True Negatives: 427
- False Positives: 156
- False Negatives: 5
- True Positives: 1

### Top K Risk Prediction Analysis

- **Top 5%**: Captures 14.3% of all injuries with 8.6% precision
- **Top 10%**: Captures 23.4% of all injuries with 7.0% precision
- **Top 15%**: Captures 39.0% of all injuries with 7.8% precision
- **Top 20%**: Captures 51.9% of all injuries with 7.8% precision

### Feature Importance

**Top 20 Most Important Features:**

1. **Fatigue Score** (0.0806) - Primary exhaustion indicator
2. **Rebounds vs Season Average** (0.0611) - Rebounding performance relative to baseline
3. **Current vs 14-Day Average** (0.0526) - Recent performance comparison
4. **Substitution Frequency** (0.0520) - In-game replacement patterns
5. **Game Day of Week** (0.0480) - Weekly scheduling impact
6. **Shots vs Season Average** (0.0434) - Shooting volume deviation
7. **Missed Shots** (0.0426) - Shooting inefficiency metric
8. **Is Back to Back** (0.0420) - Consecutive game indicator
9. **Rest Days Since Last** (0.0394) - Recovery time between games
10. **Defensive Load 30d** (0.0380) - Monthly defensive workload
11. **Shooting Efficiency Decline** (0.0349) - Performance deterioration
12. **Contact Usage Rate** (0.0347) - Physical involvement level
13. **Fouls** (0.0337) - Foul accumulation indicator
14. **Total Actions** (0.0332) - Overall activity measure
15. **Efficiency Trend 7d** (0.0327) - Short-term efficiency pattern
16. **Shooting Load 30d** (0.0322) - Monthly shooting workload
17. **BMI** (0.0299) - Body mass index impact
18. **Actions Trend 7d** (0.0275) - Recent activity changes
19. **Performance Drop 7 vs 30** (0.0261) - Performance decline comparison
20. **Turnovers** (0.0254) - Ball handling efficiency

In [12]:
# XGBoost Model

# XGBoost Model 
def train_xgboost_model(X_train_scaled, y_train, X_val_scaled, y_val, class_weights, random_state=42):
    """
    Trains XGBoost model
    """
    # Calculates scale_pos_weight for XGBoost
    neg_count = (y_train == 0).sum()
    pos_count = (y_train == 1).sum()
    scale_pos_weight = neg_count / pos_count
    
    print(f"Class distribution in training:")
    print(f"- Negative class: {neg_count:,}")
    print(f"- Positive class: {pos_count:,}")
    print(f"- Scale pos weight: {scale_pos_weight:.2f}")
    
    # Base XGBoost model 
    xgb_base = xgb.XGBClassifier(
        objective='binary:logistic',
        scale_pos_weight=scale_pos_weight,  # Handles imbalance
        random_state=random_state,
        eval_metric=['logloss', 'auc'],
        early_stopping_rounds=10,
        n_jobs=-1
    )
    
    # Hyperparameter grid
    param_grid = {
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.05, 0.1], 
        'n_estimators': [100, 200, 500],
        'subsample': [0.7, 0.8, 0.9],
        'colsample_bytree': [0.7, 0.8, 0.9],
        'reg_alpha': [0, 0.1, 0.5],  # L1 regularization
        'reg_lambda': [1, 1.5, 2]   # L2 regularization
    }
    
    # Stratified CV for imbalanced data
    cv_strategy = StratifiedKFold(n_splits=3, shuffle=True, random_state=random_state)
    
    # Grid search with PR-AUC scoring
    print("Performing hyperparameter tuning...")
    grid_search = GridSearchCV(
        estimator=xgb_base,
        param_grid=param_grid,
        scoring='average_precision',  # PR-AUC 
        cv=cv_strategy,
        n_jobs=-1,
        verbose=1,
        return_train_score=True
    )
    
    # Fit with validation set for early stopping
    grid_search.fit(
        X_train_scaled, y_train,
        eval_set=[(X_val_scaled, y_val)],
        verbose=False
    )
    
    best_model = grid_search.best_estimator_
    
    print(f"\nBest hyperparameters:")
    for param, value in grid_search.best_params_.items():
        print(f"- {param}: {value}")
    
    print(f"Best CV PR-AUC: {grid_search.best_score_:.3f}")
    
    return best_model, grid_search

# Train XGBoost model
xgb_model, xgb_grid_search = train_xgboost_model(
    X_train_scaled, y_train, 
    X_val_scaled, y_val, 
    class_weights
)

# Predictions

# Training predictions
y_train_pred_xgb = xgb_model.predict(X_train_scaled)
y_train_prob_xgb = xgb_model.predict_proba(X_train_scaled)[:, 1]

# Validation predictions  
y_val_pred_xgb = xgb_model.predict(X_val_scaled)
y_val_prob_xgb = xgb_model.predict_proba(X_val_scaled)[:, 1]

# Test predictions
y_test_pred_xgb = xgb_model.predict(X_test_scaled) 
y_test_prob_xgb = xgb_model.predict_proba(X_test_scaled)[:, 1]

# Evaluations
xgb_train_metrics = evaluate_model_performance(y_train, y_train_pred_xgb, y_train_prob_xgb, "XGBoost Training")
xgb_val_metrics = evaluate_model_performance(y_val, y_val_pred_xgb, y_val_prob_xgb, "XGBoost Validation") 
xgb_test_metrics = evaluate_model_performance(y_test, y_test_pred_xgb, y_test_prob_xgb, "XGBoost Test")

# Feature importance analysis

# Get feature importance
feature_importance_xgb = pd.DataFrame({
    'feature': selected_features,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False)

print("Top 20 Most Important Features (XGBoost):")
print(f"{'Feature':<30} {'Importance':<12}")
for idx, row in feature_importance_xgb.head(20).iterrows():
    print(f"{row['feature']:<30} {row['importance']:>10.4f}")

# Top K Risk Analysis for XGBoost
# Analyze on validation set
analyze_top_k_predictions(y_val.values, y_val_prob_xgb)


Class distribution in training:
- Negative class: 6,808
- Positive class: 167
- Scale pos weight: 40.77
Performing hyperparameter tuning...
Fitting 3 folds for each of 2187 candidates, totalling 6561 fits

Best hyperparameters:
- colsample_bytree: 0.7
- learning_rate: 0.1
- max_depth: 3
- n_estimators: 100
- reg_alpha: 0.5
- reg_lambda: 2
- subsample: 0.7
Best CV PR-AUC: 0.143

XGBOOST TRAINING Performance:
- Accuracy: 0.822
- Precision: 0.097
- Recall: 0.772
- F1-Score: 0.172
- ROC AUC: 0.884
- PR AUC: 0.261
Confusion Matrix:
    TN: 5607 | FP: 1201
    FN:   38 | TP:  129

XGBOOST VALIDATION Performance:
- Accuracy: 0.851
- Precision: 0.075
- Recall: 0.351
- F1-Score: 0.124
- ROC AUC: 0.690
- PR AUC: 0.083
Confusion Matrix:
    TN: 2158 | FP:  332
    FN:   50 | TP:   27

XGBOOST TEST Performance:
- Accuracy: 0.727
- Precision: 0.006
- Recall: 0.167
- F1-Score: 0.012
- ROC AUC: 0.396
- PR AUC: 0.011
Confusion Matrix:
    TN:  427 | FP:  156
    FN:    5 | TP:    1
Top 20 Most Importa

# Neural Network Model

### Overview

The neural network model shows early convergence with training stopping at epoch 17 due to validation loss stagnation. Despite architecture w/ dropout layers and batch normalization, the model shows some overfitting and struggles with generalization on unseen data. The training process revealed quick initial learning followed by validation performance plateau, suggesting the model reached its capacity for this dataset relatively quickly.

### Configuration

**Architecture Design:**
- Input layer: 34 features
- Hidden layer 1: 64 units with ReLU activation, 30% dropout, batch normalization
- Hidden layer 2: 32 units with ReLU activation, 30% dropout, batch normalization  
- Hidden layer 3: 16 units with ReLU activation, 20% dropout, batch normalization
- Output layer: 1 unit with sigmoid activation

**Training Configuration:**
- Training samples: 6,975
- Class weights: {0: 0.512, 1: 20.883}
- Batch size: 32
- Maximum epochs: 200
- Early stopping patience: 15 epochs
- Learning rate reduction patience: 10 epochs
- Initial learning rate: 0.001

**Training Process:**
- Total epochs trained: 17
- Best epoch: 2 (lowest validation loss)
- Training time: 11.9 seconds
- Learning rate reduced to 0.0005 at epoch 12
- Early stopping triggered at epoch 17

### Performance

**Training Set Performance**
- Accuracy: 77.1%
- Precision: 5.9%
- Recall: 56.9%
- F1-Score: 10.6%
- ROC AUC: 73.7%
- PR AUC: 7.7%

**Confusion Matrix:**
- True Negatives: 5,281
- False Positives: 1,527
- False Negatives: 72
- True Positives: 95

**Validation Set Performance**
- Accuracy: 74.3%
- Precision: 5.2%
- Recall: 44.2%
- F1-Score: 9.3%
- ROC AUC: 61.5%
- PR AUC: 5.5%

**Confusion Matrix:**
- True Negatives: 1,872
- False Positives: 618
- False Negatives: 43
- True Positives: 34

**Test Set Performance**
- Accuracy: 61.6%
- Precision: 0.9%
- Recall: 33.3%
- F1-Score: 1.7%
- ROC AUC: 44.9%
- PR AUC: 1.3%

**Confusion Matrix:**
- True Negatives: 361
- False Positives: 222
- False Negatives: 4
- True Positives: 2

### Top K Risk Prediction Analysis

- **Top 5%**: Captures 11.7% of all injuries with 7.0% precision
- **Top 10%**: Captures 20.8% of all injuries with 6.2% precision
- **Top 15%**: Captures 26.0% of all injuries with 5.2% precision
- **Top 20%**: Captures 36.4% of all injuries with 5.5% precision

### Training History Analysis

**Loss Progression:**
- Final training loss: 0.5259
- Final validation loss: 0.8868
- Best validation loss: 0.7746 (achieved at epoch 2)
- Training loss decreased consistently while validation loss increased, indicating overfitting

**Model Complexity:**
- Total parameters: 5,313
- Trainable parameters: 5,089
- Non-trainable parameters: 224 (batch normalization)
- Model size: approximately 20.8 KB

**Learning Dynamics:**
- Initial rapid improvement in first 2 epochs
- Validation performance plateau despite continued training improvements
- Learning rate reduction at epoch 12 failed to improve validation performance
- Early stopping prevented further overfitting

In [13]:
# Neural Network Architecture
def create_neural_network(input_dim, class_weights):
    """
    Creates neural network architecture
    - Input: 40 features  
    - Hidden Layer 1: 64 units, ReLU, Dropout(0.3)
    - Hidden Layer 2: 32 units, ReLU, Dropout(0.3) 
    - Hidden Layer 3: 16 units, ReLU, Dropout(0.2)
    - Output: 1 unit, Sigmoid
    """
    model = Sequential([
        # Input layer
        Dense(64, activation='relu', input_shape=(input_dim,), name='hidden_1'),
        Dropout(0.3, name='dropout_1'),
        BatchNormalization(name='batch_norm_1'),
        
        # 2nd hidden layer
        Dense(32, activation='relu', name='hidden_2'),
        Dropout(0.3, name='dropout_2'),
        BatchNormalization(name='batch_norm_2'),
        
        # 3rd hidden layer  
        Dense(16, activation='relu', name='hidden_3'),
        Dropout(0.2, name='dropout_3'),
        BatchNormalization(name='batch_norm_3'),
        
        # Output layer
        Dense(1, activation='sigmoid', name='output')
    ])
    
    # Compiles w/ class weights incorporated into loss
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall', 'AUC']
    )
    
    return model

# Convert class weights to sample weights for TensorFlow
def create_sample_weights(y, class_weights):
    """Convert class weights to sample weights for TensorFlow training"""
    sample_weights = np.where(y == 1, class_weights[1], class_weights[0])
    return sample_weights

# Training setup
print("Setting up Neural Network training...")
print(f"Input features: {X_train_tf.shape[1]}")
print(f"Training samples: {X_train_tf.shape[0]:,}")
print(f"Class weights: {class_weights}")

# Creates sample weights for training
train_sample_weights = create_sample_weights(y_train_tf, class_weights)
val_sample_weights = create_sample_weights(y_val_tf, class_weights)

print(f"Sample weights created. Shape: {train_sample_weights.shape}")
print(f"Sample weight distribution: {np.unique(train_sample_weights, return_counts=True)}")

# Builds the model
nn_model = create_neural_network(X_train_tf.shape[1], class_weights)

# Model summary
print("\nNeural Network Architecture:")
nn_model.summary()

# Callbacks for training
callbacks_list = [
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1,
        mode='min'
    ),
    ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.5,
        patience=10,
        min_lr=1e-7,
        verbose=1,
        mode='min'
    ),
    ModelCheckpoint(
        filepath=f'../data/processed/{MODEL_NAME}_best_nn.h5',
        monitor='val_loss',
        save_best_only=True,
        verbose=1,
        mode='min'
    )
]

# Training parameters
BATCH_SIZE = 32
EPOCHS = 200
VALIDATION_SPLIT = 0.0  # Separate validation set

print(f"\nTraining Configuration:")
print(f"- Batch size: {BATCH_SIZE}")
print(f"- Max epochs: {EPOCHS}")
print(f"- Early stopping patience: 15")
print(f"- Learning rate reduction patience: 10")

# Trains the model
print("\nTraining Started...")
start_time = time.time()

history = nn_model.fit(
    X_train_tf, y_train_tf,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_val_tf, y_val_tf, val_sample_weights),
    sample_weight=train_sample_weights,
    callbacks=callbacks_list,
    verbose=1,
    shuffle=True
)

training_time = time.time() - start_time
print(f"\nTraining completed in {training_time:.1f} seconds ({training_time/60:.1f} minutes)")

# Training predictions
y_train_prob_nn = nn_model.predict(X_train_tf, batch_size=BATCH_SIZE, verbose=0).flatten()
y_train_pred_nn = (y_train_prob_nn > 0.5).astype(int)

# Validation predictions  
y_val_prob_nn = nn_model.predict(X_val_tf, batch_size=BATCH_SIZE, verbose=0).flatten()
y_val_pred_nn = (y_val_prob_nn > 0.5).astype(int)

# Test predictions
y_test_prob_nn = nn_model.predict(X_test_tf, batch_size=BATCH_SIZE, verbose=0).flatten()
y_test_pred_nn = (y_test_prob_nn > 0.5).astype(int)

# Evaluations
nn_train_metrics = evaluate_model_performance(y_train_tf.astype(int), y_train_pred_nn, y_train_prob_nn, "Neural Network Training")
nn_val_metrics = evaluate_model_performance(y_val_tf.astype(int), y_val_pred_nn, y_val_prob_nn, "Neural Network Validation") 
nn_test_metrics = evaluate_model_performance(y_test_tf.astype(int), y_test_pred_nn, y_test_prob_nn, "Neural Network Test")

# Top K Risk Analysis for Neural Network
analyze_top_k_predictions(y_val_tf.astype(int), y_val_prob_nn)

# Training history summary
print("Training History Summary:")
print(f"- Total epochs trained: {len(history.history['loss'])}")
print(f"- Best epoch (lowest val_loss): {np.argmin(history.history['val_loss']) + 1}")
print(f"- Final training loss: {history.history['loss'][-1]:.4f}")
print(f"- Final validation loss: {history.history['val_loss'][-1]:.4f}")
print(f"- Best validation loss: {min(history.history['val_loss']):.4f}")

if 'val_auc' in history.history:
    print(f"- Final validation AUC: {history.history['val_auc'][-1]:.4f}")
    print(f"- Best validation AUC: {max(history.history['val_auc']):.4f}")

# Model complexity summary
total_params = nn_model.count_params()
trainable_params = sum([tf.keras.backend.count_params(w) for w in nn_model.trainable_weights])

print(f"\nModel Complexity:")
print(f"- Total parameters: {total_params:,}")
print(f"- Trainable parameters: {trainable_params:,}") 
print(f"- Model size: ~{total_params * 4 / 1024:.1f} KB (float32)")

Setting up Neural Network training...
Input features: 34
Training samples: 6,975
Class weights: {0: 0.512264982373678, 1: 20.88323353293413}
Sample weights created. Shape: (6975,)
Sample weight distribution: (array([ 0.51226498, 20.88323353]), array([6808,  167], dtype=int64))

Neural Network Architecture:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ hidden_1 (Dense)                │ (None, 64)             │         2,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_norm_1                    │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_2 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_norm_2                    │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_3 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_norm_3                    │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,313 (20.75 KB)

 Trainable params: 5,089 (19.88 KB)

 Non-trainable params: 224 (896.00 B)


Training Configuration:
- Batch size: 32
- Max epochs: 200
- Early stopping patience: 15
- Learning rate reduction patience: 10

Training Started...
Epoch 1/200
203/218 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.5366 - accuracy: 0.5131 - loss: 0.9349 - precision: 0.0329 - recall: 0.5834
Epoch 1: val_loss improved from None to 0.79834, saving model to ../data/processed/nba_injury_predictor_v1_best_nn.h5


218/218 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - AUC: 0.5545 - accuracy: 0.5204 - loss: 0.8291 - precision: 0.0285 - recall: 0.5749 - val_AUC: 0.5760 - val_accuracy: 0.7713 - val_loss: 0.7983 - val_precision: 0.0364 - val_recall: 0.2597 - learning_rate: 0.0010
Epoch 2/200
193/218 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.6263 - accuracy: 0.5467 - loss: 0.7682 - precision: 0.0401 - recall: 0.6602
Epoch 2: val_loss improved from 0.79834 to 0.77461, saving model to ../data/processed/nba_injury_predictor_v1_best_nn.h5


218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.6371 - accuracy: 0.5568 - loss: 0.7017 - precision: 0.0344 - recall: 0.6467 - val_AUC: 0.6141 - val_accuracy: 0.7425 - val_loss: 0.7746 - val_precision: 0.0521 - val_recall: 0.4416 - learning_rate: 0.0010
Epoch 3/200
193/218 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.6650 - accuracy: 0.5943 - loss: 0.7400 - precision: 0.0466 - recall: 0.6958
Epoch 3: val_loss did not improve from 0.77461
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.6527 - accuracy: 0.5950 - loss: 0.6918 - precision: 0.0376 - recall: 0.6467 - val_AUC: 0.6053 - val_accuracy: 0.7164 - val_loss: 0.7788 - val_precision: 0.0435 - val_recall: 0.4026 - learning_rate: 0.0010
Epoch 4/200
212/218 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.6549 - accuracy: 0.6014 - loss: 0.7417 - precision: 0.0433 - recall: 0.6452
Epoch 4: val_loss did not improve from 0.77461
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - AUC: 0.6800 - accuracy: 0.6047 - loss: 0.6595 - precision: 0.0405 - recall: 0